In [ ]:
# import numpy as np
# from datetime import datetime 
# import matplotlib.pyplot as plt
# import pandas as pd
# from scipy import stats
# import plotly.express as px
# plt.style.use('ggplot')   
# %matplotlib inline

In [ ]:
# pl_pool=pd.read_csv('plr_pool.csv', parse_dates = ['last_visit'])


## RFM аббревиатура (англ. Recency Frequency Monetary — давность, частота, деньги) 

Определяет три группы:

**Recency** (давность, пер.: новизна, свежесть) — давность сделки: чем меньше времени прошло с момента последней активности клиента, тем больше вероятность, что он повторит действие.

**Frequency** (частота) — количество сделок: чем больше каких-либо действий совершил клиент, тем больше вероятность того, что он его повторит в будущем.

**Monetary** (деньги) — сумма сделок: чем больше денег было потрачено, тем больше вероятность того, что он сделает заказ[

#### Другие анализы

ABC-анализ
метод, позволяющий классифицировать ресурсы фирмы по степени их важности.

`А — наиболее ценные, 20 % — ассортимента (номенклатура); 80 % — продаж`

`В — промежуточные, 30 % — ассортимента; 15 % — продаж`

`С — наименее ценные, 50 % — ассортимента; 5 % — продаж `

XYZ-анализ
анализ, где ресурсы компании классифицируют на три категории по характеру потребления и точности (волатильности) предсказания потребления в течение определённого времени

    X — имеют стабильный, с минимальными колебаниями спрос. Эти ресурсы нужно «держать под рукой»
    Y — имеют нестабильный, но предсказуемый спрос. Этим ресурсам нужно найти баланс.
    Z — имеют крайне волатильный спрос. Эти ресурсы лучше сбывать по предзаказу.

VEN-анализ
........
FMR-анализ
........

### Обозначаем Крайний день

In [ ]:
# end_date = datetime.strptime("01/1/2025", "%d/%m/%Y")

### вычисляем как давно был гость

In [ ]:
# pl_pool['Recency'] = (end_date-pl_pool.last_visit).dt.days

In [ ]:
# pl_pool['Recency']

### Выбираем, что у нас будет 'Frequency' и 'Monetary'

In [ ]:
# # rfm_df = pl_pool[['Recency','visits','res_sum']]
# rfm_df = pl_pool[['Recency','visits','bets_sum']]
# rfm_df.columns = ['Recency', 'Frequency', 'Monetary']

In [ ]:
# rfm_df

In [ ]:
# rfm_df[['Recency', 'Frequency', 'Monetary']].hist(figsize=(20,10), bins=50);

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

#### каждую категорию делим на три части

In [ ]:
# rfm_df['r_categores'] = pd.qcut(rfm_df['Recency'], q=3, labels=['1', '2', '3'])
# rfm_df['f_categores'] = pd.qcut(rfm_df['Frequency'], q=3, labels=['3', '2', '1'])
# rfm_df['m_categores'] = pd.qcut(rfm_df['Monetary'], q=3, labels=['1', '2', '3'])



In [ ]:
# rfm_df

### Склеиваем

In [ ]:
# rfm_df['RFM_RES'] = rfm_df['r_categores'].astype(str) +rfm_df['f_categores'].astype(str) +rfm_df['m_categores'].astype(str) 

In [ ]:
# rfm_df

In [ ]:
# px.scatter_3d(
#     rfm_df,
#     x='Recency',
#     y='Frequency',
#     z='Monetary',
#     color='RFM_RES'
# ).show(renderer='browser')

### Делаем красиво:

Близкие точки раздвигаем.

Дальние приближаем.

Т.е. логарифмируем

In [ ]:
# rfm_df[['Recency', 'Frequency', 'Monetary'] ] = rfm_df[['Recency', 'Frequency', 'Monetary']].apply(lambda x: np.log(x+1))

In [ ]:
# px.scatter_3d(
#     rfm_df,
#     x='Recency',
#     y='Frequency',
#     z='Monetary',
#     color='RFM_RES'
# ).show(renderer='browser')

#### Расмотрим основной Дата Фрейм, посмотрим какие данные лучше отБокса_коксить. (отлогарифмировать)

In [ ]:
# pl_pool.head()

In [ ]:
# pl_pool[['visits','res_sum','Recency','hands_sum','bets_sum' ]].hist(figsize=(20,10),bins = 50);

In [ ]:
# pl_pool[['visits','Recency','hands_sum','bets_sum' ]] = pl_pool[['visits','Recency','hands_sum','bets_sum' ]].apply(lambda x:np.log(x+1) )

In [ ]:
# pl_pool[['visits','res_sum','Recency','hands_sum' ,'bets_sum' ]].hist(figsize=(20,10),bins = 50);

### Нормируем данные, чтобы можно измерЯть расстояние

#### Для положительных сделаем мин макс нормирование

In [ ]:
# def min_max_scaler(serias):
#     return (serias- serias.min())/(serias.max()-serias.min()) 

In [ ]:
# pl_pool[['visits','Recency','hands_sum' ,'bets_sum' ]] = min_max_scaler(pl_pool[['visits','Recency','hands_sum' ,'bets_sum' ]])

#### Для тех, которые могут быть отрицательным значением сделаем нормальное нормирование

In [ ]:
# def norm_scaler(serias):
#     return (serias- serias.mean())/serias.std() 

In [ ]:
# pl_pool['res_sum'] = norm_scaler(pl_pool['res_sum'])

In [ ]:
# pl_pool[['visits','res_sum','Recency','hands_sum' ,'bets_sum' ]].hist(figsize=(20,10),bins = 50);


In [ ]:
# px.scatter_3d(
#     pl_pool,
#     x='visits',
#     y='bets_sum',
#     z='Recency',
#     color='res_sum'
# ).show(renderer='browser')

### Решения из коробочки

#### K ближние соседи

In [ ]:
    # from sklearn.cluster import KMeans
    # from sklearn.preprocessing import StandardScaler


In [ ]:
# # собираем дата фрейм без лишних колонок
# clear_pl_pool = pl_pool[['visits', 'bets_sum', 'Recency']] 



# # определяем и обозначаем кол-во кластеров насколько хотим делить
# kmeans = KMeans(n_clusters=3)
# kmeans.fit(clear_pl_pool)
# pl_pool['cluster'] = kmeans.labels_
# # clear_pl_pool['cluster'] = kmeans.labels_
# cluster_centers_scaled = kmeans.cluster_centers_

In [ ]:
# cluster_centers_scaled


In [ ]:
# pl_pool


In [ ]:
# px.scatter_3d(
#     pl_pool,
#     x='visits',
#     y='bets_sum',
#     z='Recency',
#     color='cluster'
# ).show(renderer='browser')

### DBSCAN
нужно определять eps это радиус окресности вокруг точки

min_samples - минимальное кол-во точек в окресности, чтобы точку считать основной

In [ ]:
# from sklearn.cluster import DBSCAN

In [ ]:
# dbscan = DBSCAN(eps=0.2, min_samples=7)
# clusters = dbscan.fit_predict(clear_pl_pool)
# pl_pool['DBCluster'] = clusters

In [ ]:
# px.scatter_3d(
#     pl_pool,
#     x='visits',
#     y='bets_sum',
#     z='Recency',
#     color='DBCluster'
# ).show(renderer='browser')